In [1]:
import findspark
findspark.init()


from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.master("local[4]") \
        .appName("Joins").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/12 14:08:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Customer DataFrame

In [4]:
customer_data = [(1,'manish','patna',"30-05-2022"),
(2,'vikash','kolkata',"12-03-2023"),
(3,'nikita','delhi',"25-06-2023"),
(4,'rahul','ranchi',"24-03-2023"),
(5,'mahesh','jaipur',"22-03-2023"),
(6,'prantosh','kolkata',"18-10-2022"),
(7,'raman','patna',"30-12-2022"),
(8,'prakash','ranchi',"24-02-2023"),
(9,'ragini','kolkata',"03-03-2023"),
(10,'raushan','jaipur',"05-02-2023")]


customer_schema = StructType([StructField("customer_id", IntegerType(), True),
                              StructField("customer_name", StringType(), True),
                              StructField("address", StringType(), True),
                              StructField("date_of_joining", StringType(), True)])

customer_df = spark.createDataFrame(customer_data, customer_schema)

print("Customer DataFrame:")
customer_df.show(5)

Customer DataFrame:
+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          1|       manish|  patna|     30-05-2022|
|          2|       vikash|kolkata|     12-03-2023|
|          3|       nikita|  delhi|     25-06-2023|
|          4|        rahul| ranchi|     24-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|
+-----------+-------------+-------+---------------+
only showing top 5 rows



In [5]:
sales_data = [(1,22,10,"01-06-2022"),
(1,27,5,"03-02-2023"),
(2,5,3,"01-06-2023"),
(5,22,1,"22-03-2023"),
(7,22,4,"03-02-2023"),
(9,5,6,"03-03-2023"),
(2,1,12,"15-06-2023"),
(1,56,2,"25-06-2023"),
(5,12,5,"15-04-2023"),
(11,12,76,"12-03-2023")]

# sales_schema=['customer_id','product_id','quantity','date_of_purchase']

sales_schema = StructType([StructField("customer_id", IntegerType(), True),
                           StructField("product_id", IntegerType(), True),
                           StructField("quantity", IntegerType(), True),
                           StructField("date_of_purchase", StringType(), True)])

sales_df = spark.createDataFrame(sales_data, sales_schema)

sales_df.show(5)

+-----------+----------+--------+----------------+
|customer_id|product_id|quantity|date_of_purchase|
+-----------+----------+--------+----------------+
|          1|        22|      10|      01-06-2022|
|          1|        27|       5|      03-02-2023|
|          2|         5|       3|      01-06-2023|
|          5|        22|       1|      22-03-2023|
|          7|        22|       4|      03-02-2023|
+-----------+----------+--------+----------------+
only showing top 5 rows



Types of Join in Spark

In [6]:
# Inner Join

inner_join_df = customer_df.join(sales_df, customer_df.customer_id == sales_df.customer_id, "inner")
inner_join_df.show()

+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        12|       5|      15

In [8]:
#Outer (Full Outer Join)

full_join_df = customer_df.join(sales_df, customer_df.customer_id == sales_df.customer_id, "full")
print("Full Join Count", full_join_df.count())
full_join_df.show()

Full Join Count 15
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          3|       nikita|  delhi|     25-06-2023|       NULL|      NULL|    NULL|            NULL|
|          4|        rahul| ranchi|     24-03-2023|       NULL|      NUL

In [9]:
#Left Join

left_join_df = customer_df.join(sales_df, customer_df.customer_id == sales_df.customer_id, "left")
print("left join record count :", left_join_df.count())
left_join_df.show()

left join record count : 14
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        56|       2|      25-06-2023|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          2|       vikash|kolkata|     12-03-2023|          2|         1|      12|      15-06-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          3|       nikita|  delhi|     25-06-2023|       NULL|      NULL|    NULL|            NULL|
|          4|        rahul| ranchi|     24-03-2023|       NULL|

In [10]:
#Right Join

right_join_df = customer_df.join(sales_df, customer_df.customer_id == sales_df.customer_id, "right")
print("right join record count", right_join_df.count())
right_join_df.show()

right join record count 10
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|customer_id|customer_name|address|date_of_joining|customer_id|product_id|quantity|date_of_purchase|
+-----------+-------------+-------+---------------+-----------+----------+--------+----------------+
|          1|       manish|  patna|     30-05-2022|          1|        22|      10|      01-06-2022|
|          1|       manish|  patna|     30-05-2022|          1|        27|       5|      03-02-2023|
|          5|       mahesh| jaipur|     22-03-2023|          5|        22|       1|      22-03-2023|
|          2|       vikash|kolkata|     12-03-2023|          2|         5|       3|      01-06-2023|
|          9|       ragini|kolkata|     03-03-2023|          9|         5|       6|      03-03-2023|
|          7|        raman|  patna|     30-12-2022|          7|        22|       4|      03-02-2023|
|          1|       manish|  patna|     30-05-2022|          1| 

In [14]:
# Left Anti Join  -- Shows records from first table which are not available in second table

left_anti_join_df = customer_df.join(sales_df, customer_df.customer_id == sales_df.customer_id, "left_anti").sort("customer_id")
left_anti_join_df.show()

+-----------+-------------+-------+---------------+
|customer_id|customer_name|address|date_of_joining|
+-----------+-------------+-------+---------------+
|          3|       nikita|  delhi|     25-06-2023|
|          4|        rahul| ranchi|     24-03-2023|
|          6|     prantosh|kolkata|     18-10-2022|
|          8|      prakash| ranchi|     24-02-2023|
|         10|      raushan| jaipur|     05-02-2023|
+-----------+-------------+-------+---------------+



In [20]:
# CrossJoin

customer_df.crossJoin(sales_df).count()

100

In [21]:
spark.stop()